Install Necessary Libarary

In [11]:
%pip install --quiet ipywidgets langchain langchain_openai


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


```mermaid
graph LR
A[Document] --> |Split| B[Chunks]
    B --> |Create Embedding| C[Vector Store]
D[User] --> E(Query)
    E --> |Create Embedding| F(Similarity Search)
    F --> C
    E --> G[LLMS]
    C --> G
    
```

Import OpenAI API Key

In [12]:
import os
f = open('openaikey.txt')
os.environ["OPENAI_API_KEY"]=f.read()

Import Document and Split into Chuncks

In [13]:
from langchain_community.document_loaders.pdf import PyPDFLoader
pyPDFLoader = PyPDFLoader("resources/DesignDocumentStandards.pdf")
data = pyPDFLoader.load()

Split using Text Splitter

In [14]:
from langchain_text_splitters import PythonCodeTextSplitter
textSplitter = PythonCodeTextSplitter(chunk_size=10000,chunk_overlap=3000)
pages= pyPDFLoader.load_and_split(text_splitter=textSplitter)

Print Relevant information

In [15]:
print(f"Total Documents Length before text split {len(data)}")
print(f"Total Documents after the text split {len(pages)}")


Total Documents Length before text split 6
Total Documents after the text split 6


Initiate Embedding

In [16]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embedding = OpenAIEmbeddings()
db = FAISS.from_documents(documents=pages,embedding=embedding)
print(db.index.ntotal)

6


Try executing  a query

In [17]:
query = "What is this document about ? "
result = db.similarity_search(query=query)


Explore results

In [18]:
print(result)

[Document(metadata={'source': 'resources/DesignDocumentStandards.pdf', 'page': 0}, page_content='DESIGN DOCUMENT \nSTANDARDS'), Document(metadata={'source': 'resources/DesignDocumentStandards.pdf', 'page': 1}, page_content='Contents  \n1. Introduction  ................................ ................................ ................................ ................................ ...........  2 \n1.1. Purpose  ................................ ................................ ................................ ................................ ..........  2 \n1.2. Scope  ................................ ................................ ................................ ................................ .............  2 \n2. Document Structure  ................................ ................................ ................................ ..............................  2 \n2.1. Title  ................................ ................................ ................................ ....

Create a Chat bot

In [19]:
from langchain.chains.question_answering.chain import load_qa_chain
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

openAI = ChatOpenAI(temperature=0)

chain = load_qa_chain(openAI,chain_type="stuff")
query = "How many must have section should be present in the design document and what are they ?"
docs = db.similarity_search(query)
response = chain.invoke(input={"input_documents":docs, "question":query})

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  warn_deprecated(


Explore result

In [20]:
print(response['output_text'])

There are six "must-have" sections that should be present in the design document:

1. Summary
2. Scenarios
3. Assumptions
4. Out of Scope
5. Logical View / Affected component view
6. Reference Table


Create a Chat Component

In [ ]:
from IPython.display import display
import ipywidgets as widgets

qa = ConversationalRetrievalChain.from_llm(openAI, db.as_retriever())

chat_history = []

def on_sumbit(_):
    query = input_box.value
    input_box.value=""
    print(chat_history)
    if query.lower() == 'exit':
        print("Thank you for chatting with me")
        return
    
    result = qa.invoke(input={"question":query,"chat_history":chat_history})
    chat_history.append((query, result['answer']))
   
    display(widgets.HTML(f"Number of matches found : {len(result)}"))
    display(widgets.HTML(f"<b>User Query</b> : {query}"))
    display(widgets.HTML(f'<b><font color="blue">Chat Bot :</font></b> {result["answer"]}'))

print("Welcome to the chatbot")
input_box = widgets.Text(placeholder="Enter the Question")
input_box.on_submit(on_sumbit)

display(input_box)

Welcome to the chatbot


/var/folders/df/4n57lpzd4z986szq__z25w840000gn/T/ipykernel_53057/2205493192.py:25: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_box.on_submit(on_sumbit)


Text(value='', placeholder='Enter the Question')

[]


HTML(value='Number of matches found : 3')

...

HTML(value='<b><font color="blue">Chat Bot :</font></b> The summary section of your document should include a …

[('How my summary section be', 'The summary section of your document should include a standard requirement of the epic, such as a specific feature or change being addressed, and elaborate on the business driver behind it. This section should briefly discuss the importance of the requirement and the motivation for the engineering team to conform to the understanding of the feature. It should highlight the significance of the feature within the context of the project or system being developed.')]


HTML(value='Number of matches found : 3')

HTML(value='<b>User Query</b> : what was my last question')

HTML(value='<b><font color="blue">Chat Bot :</font></b> I don\'t know.')

In [22]:
%pip install PyPDF2


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Review Document

In [23]:
from IPython.display import display
import ipywidgets as widgets

chat_history = []
qa = ConversationalRetrievalChain.from_llm(openAI, db.as_retriever())
pyPDFLoader = PyPDFLoader("resources/TechnicalDocument.pdf")
documents = pyPDFLoader.load()
content = documents[0].page_content

query=f"""
Given the knowledge base you have help me reivew the content mentioned enclosed between <start> and <end> tag and provide your review comments
<start>
{content}
<end>

Validate if all the necessary sections are present, specify which sections are missing and how the content should be. Finally review the content and provide the feedback checking the compliance.
Also give the final verdict as "Approved" or "Rejected"
"""
print(query)

#result = qa.invoke(input={"question":query,"chat_history":chat_history})
#chat_history.append((query, result['answer']))
openAI.e
print(result["answer"])



Given the knowledge base you have help me reivew the content mentioned enclosed between <start> and <end> tag and provide your review comments
<start>
Technical Design – Abrupt implementation  
 
1. Summary  
This document explains the details of the implementation of the solution 
provided in the previous document.  
<end>

Validate if all the necessary sections are present, specify which sections are missing and how the content should be. Finally review the content and provide the feedback checking the compliance.
Also give the final verdict as "Approved" or "Rejected"



AttributeError: 'ChatOpenAI' object has no attribute 'e'